# STEP1: EDA 基本数据分析
目前是没有对数据做特别多的数据分析，待后期完善

In [3]:
### STEP 1:EDA
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import matplotlib as mpl
import seaborn as sns
%matplotlib inline


# common prefix
TRAIN_PREFIX = "../train/"
TEST_PREFIX = "../B/"
FEATURE_PREFIX = "../feature/"

#Train-dataset
BANK_DETAIL_TRAIN_FILE = TRAIN_PREFIX + "bank_detail_train.csv"
BILL_DETAIL_TRAIN_FILE = TRAIN_PREFIX + "bill_detail_train.csv"
BROWSE_HISTORY_TRAIN_FILE = TRAIN_PREFIX +"browse_history_train.csv"
LOANTIME_TRAIN_FILE = TRAIN_PREFIX + "loantime_train.csv"
OVERDUE_TRAIN_FILE = TRAIN_PREFIX + "overdue_train.csv"
USERINFO_TRAIN_FILE = TRAIN_PREFIX + "userinfo_train.csv"

#Test-dataset
BANK_DETAIL_TEST_FILE = TEST_PREFIX + "bank_detail_B.csv"
BILL_DETAIL_TEST_FILE = TEST_PREFIX + "bill_detail_B.csv"
BROWSE_HISTORY_TEST_FILE = TEST_PREFIX + "browse_history_B.csv"
LOANTIME_TEST_FILE = TEST_PREFIX + "loantime_B.csv"
USERINFO_TEST_FILE = TEST_PREFIX + "userinfo_B.csv"

    

def analyseBankDetail():
    print("analyse bank_detail sheet completed")
    
def analyseBrowseHistory():
    print("analyse browse_history sheet completed")

def analyseBillDetail():
    print("analyse browse_history sheet completed")


def setEnv():
    mpl.rcParams['font.sans-serif'] = ['SimHei']
    mpl.rcParams['font.serif'] = ['SimHei']

    p = sns.color_palette()
    sns.set_style("darkgrid",{"font.sans-serif":['simhei', 'Arial']})
    print("init completed")

setEnv()


init completed


# 1.user_info相关分析


In [4]:
#def analyseUserInfo():
user_info = pd.read_csv(USERINFO_TRAIN_FILE)
print(user_info.shape)
user_info_test = pd.read_csv(USERINFO_TEST_FILE)

user_info_test = user_info_test.head()
print(user_info_test.shape)


FileNotFoundError: File b'../train/userinfo_train.csv' does not exist

In [ ]:
import lightgbm as lgbm
from scipy import sparse as ssp
from sklearn.model_selection import StratifiedKFold
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
for s in xrange(16):
    print(s)

# 2. bill_detail相关分析

In [ ]:
#def analyseBillDetail():
detail = pd.read_csv(BILL_DETAIL_TRAIN_FILE,
                     header=0,
                     names=['用户标识','时间','银行标识','上期账单金额','上期还款金额','信用卡额度',
                           '本期账单余额','本期账单最低还款额','消费笔数','本期账单金额','调整金额',
                          '循环利息','可用余额','预借现金额度','还款状态']).drop_duplicates()
print(detail.shape)


In [ ]:
detail = pd.read_csv(BILL_DETAIL_TRAIN_FILE,
                     header=0,
                     names=['用户标识','时间','银行标识','上期账单金额','上期还款金额','信用卡额度',
                           '本期账单余额','本期账单最低还款额','消费笔数','本期账单金额','调整金额',
                          '循环利息','可用余额','预借现金额度','还款状态']).drop_duplicates()

overdue = pd.read_csv(OVERDUE_TRAIN_FILE,header=0,names=['用户标识','标签'])
#overdue[overdue['标签']==0].shape[0]/overdue[overdue['标签']==1].shape[0]
detail = pd.merge(overdue, detail, how = 'left', on = '用户标识')

feature_basic = detail
feature_basic['时间'] = feature_basic['时间'] // 86400
feature_basic['上期未还款金额'] = feature_basic['上期账单金额'] - feature_basic['上期还款金额']
feature_basic['相邻两期账单金额差'] = feature_basic['本期账单金额'] - feature_basic['上期账单金额']
feature_basic['本期还款总额'] = feature_basic['上期账单金额'] - feature_basic['上期还款金额'] + feature_basic['本期账单金额'] - feature_basic['调整金额'] + feature_basic['循环利息']
feature_basic['已经使用信用卡额度'] = feature_basic['信用卡额度'] - feature_basic['可用余额']


# gb = feature_basic.loc[:,['用户标识' ,'相邻两期账单金额差', '本期还款总额', '已经使用信用卡额度']].groupby(["用户标识"], as_index=False)
# feature_agg = gb.agg({'mean', 'var', 'max', 'min', 'median'}).add_prefix('放款前_').reset_index()

In [ ]:
feature_basic[feature_basic['标签']==1]

In [ ]:
loantime = pd.read_csv(LOANTIME_TEST_FILE,header=0,names=['用户标识','放款时间'])
loantime['放款时间']=loantime['放款时间']//86400

bill_detail=pd.read_csv(BILL_DETAIL_TEST_FILE,
                 header=0,
                 names=['用户标识','时间','银行标识','上期账单金额','上期还款金额','信用卡额度',
                       '本期账单余额','本期账单最低还款额','消费笔数','本期账单金额','调整金额',
                      '循环利息','可用余额','预借现金额度','还款状态']).drop_duplicates()
bill_detail['时间']=bill_detail['时间']//86400

bill_detail = pd.merge(bill_detail, loantime,how='inner', on = "用户标识")
d=bill_detail[(bill_detail['时间']>0)]


In [5]:
feature = loantime
group_by=d[(d['时间']<=d['放款时间'])].loc[:,['用户标识', '上期账单金额', '上期还款金额','信用卡额度','本期账单余额','本期账单最低还款额',
                                      '消费笔数','本期账单金额','调整金额','循环利息','可用余额','预借现金额度','还款状态']].groupby(["用户标识"],
                                                                                                     as_index=False)

bill_beforeloan_std = group_by.agg(['std','count'])
bill_beforeloan_std
#
#bill_beforeloan_std.columns = [ '放款前上期账单金额std', '放款前上期还款金额std','放款前信用卡额度std','放款前本期账单余额std',
#                         '放款前本期账单最低还款额std','放款前消费笔数std','放款前本期账单金额std','放款前调整金额std','放款前循环利息std',
#                         '放款前可用余额std','放款前预借现金额度std','放款前还款状态std']
#
#feature=pd.merge(feature, bill_beforeloan_std,how='left', on = "用户标识")
#feature

NameError: name 'loantime' is not defined

In [6]:
# # 增加放款给用户之前，用户信用卡账单的各种统计信息sum count max min mean std var等
d=bill_detail[(bill_detail['时间']>0)]
prefix = '放款前'
condition = (d['时间']<=d['放款时间'])
#print(d[condition].shape)
group_by=d[condition].loc[:,['用户标识', '时间', '上期账单金额', '上期还款金额','信用卡额度','本期账单余额','本期账单最低还款额',
                                  '消费笔数','本期账单金额','调整金额','循环利息','可用余额','预借现金额度','还款状态']].groupby(["用户标识","时间"],
                                                                                                 as_index=False)
group_by.count()


NameError: name 'bill_detail' is not defined

# 3. browse_history_detail

In [7]:
browse_detail = pd.read_csv(BROWSE_HISTORY_TRAIN_FILE,header=0,
                        names=['用户标识','浏览时间','浏览行为数据','浏览子行为编号'])
print(browse_detail.shape)
browse_detail_drop_duplicates = browse_detail.drop_duplicates()
print(browse_detail_drop_duplicates.shape)

browse_detail = pd.read_csv(BROWSE_HISTORY_TEST_FILE,header=0,
                        names=['用户标识','浏览时间','浏览行为数据','浏览子行为编号'])
print(browse_detail.shape)
browse_detail_drop_duplicates = browse_detail.drop_duplicates()
print(browse_detail_drop_duplicates.shape)

FileNotFoundError: File b'../train/browse_history_train.csv' does not exist

In [8]:
loantime = pd.read_csv(LOANTIME_TEST_FILE,header=0,names=['用户标识','放款时间'])
loantime['放款时间']=loantime['放款时间']//86400
browse_detail = pd.read_csv(BROWSE_HISTORY_TEST_FILE,header=0,
                    names=['用户标识','浏览时间','浏览行为数据','浏览子行为编号']).drop_duplicates()
browse_detail['浏览时间']=browse_detail['浏览时间']//86400

feature=loantime
d = browse_detail
d = pd.get_dummies(d,columns=['浏览行为数据', '浏览子行为编号'])
gb = d.groupby(["用户标识"], as_index = False)
browse_agg = gb.agg(['count','sum','std', 'var', 'median', 'mean'])
feature=pd.merge(feature, browse_agg,how='left', on = "用户标识")
feature = feature.fillna(0)
feature

#----------------------------------------放款前特征统计------------------------------------------#
#统计放款前用户浏览子行为总数以及浏览行为数据总和
# gb=d[(d['浏览时间']<=d['放款时间'])].groupby(["用户标识"],as_index=False)
# x1=gb['浏览行为数据'].agg({'放款前浏览行为数据sum' : 'sum','放款前浏览行为数据max' : 'max','放款前浏览行为数据mean' : 'mean'
#                     ,'放款前浏览行为数据min' : 'min','放款前浏览行为数据std' : 'std','放款前浏览行为数据var' : 'var'})
# xx=gb['浏览子行为编号'].apply(lambda x:np.unique(x).size)
# x2=gb['浏览子行为编号'].agg({'放款前浏览子行为编号count' : 'count'})
# x2['放款前浏览子行为编号计数（去重）']=xx

# feature=pd.merge(feature, x1,how='left', on = "用户标识")
# feature=pd.merge(feature, x2,how='left', on = "用户标识")

# #统计放款前用户浏览子行为个类别统计信息
# d=pd.get_dummies(d,columns=['浏览子行为编号'])#22919547 rows × 14 columns
# gb=d[(d['浏览时间']<=d['放款时间'])].groupby(["用户标识"],as_index=False)
# d1 = pd.get_dummies(d,columns=['浏览行为数据', '浏览子行为编号'])
# d1


FileNotFoundError: File b'../B/loantime_B.csv' does not exist

# 4. bank_detail

In [9]:
bank_detail = pd.read_csv(BANK_DETAIL_TRAIN_FILE).rename(index=str,
                                                        columns={"new_user_id": "用户标识","flag1": "流水时间",
                                                                 "flag2":"交易类型","flag3":"交易金额","flag4":"工资收入标记"})
print(bank_detail.shape)
bank_detail_drop_duplicates = bank_detail.drop_duplicates()
print(bank_detail_drop_duplicates.shape)
bank_detail = pd.read_csv(BANK_DETAIL_TEST_FILE).rename(index=str,
                                                        columns={"new_user_id": "用户标识","flag1": "流水时间",
                                                                 "flag2":"交易类型","flag3":"交易金额","flag4":"工资收入标记"})
print(bank_detail.shape)
bank_detail_drop_duplicates = bank_detail.drop_duplicates()
print(bank_detail_drop_duplicates.shape)

FileNotFoundError: File b'../train/bank_detail_train.csv' does not exist

In [10]:
# features_all_train_test_20181106.csv analysis
browse_detail = pd.read_csv("../feature/features_all_train_test_20181106.csv"


SyntaxError: unexpected EOF while parsing (<ipython-input-10-4902b8445ccd>, line 2)

In [11]:
#read bank_detail & loantime
loantime = pd.read_csv(LOANTIME_TEST_FILE,header=0,names=['用户标识','放款时间'])
loantime['放款时间']=loantime['放款时间']//86400
bank_detail=pd.read_csv(BANK_DETAIL_TEST_FILE).rename(index=str,
                                                    columns={"new_user_id": "用户标识","flag1": "流水时间",
                                                             "flag2":"交易类型","flag3":"交易金额","flag4":"工资收入标记"}).drop_duplicates()
bank_detail['流水时间']=bank_detail['流水时间']//86400

bank_detail = pd.merge(bank_detail, loantime,how='inner', on = "用户标识")

FileNotFoundError: File b'../B/loantime_B.csv' does not exist

In [12]:

feature=loantime
d=bank_detail
#----------------------------------------放款前特征统计------------------------------------------#
t=d[(d['流水时间']<=d['放款时间'])]#5684742
gb1=t[(t['交易类型']==0)].groupby(["用户标识"],as_index=False)#收入统计
gb2=t[(t['交易类型']==1)].groupby(["用户标识"],as_index=False)#支出统计
t_income = t[(t['交易类型']==0)].loc[:,['用户标识', '流水时间', '交易金额']].rename(index = str, columns={'交易金额':'收入'})
t_expend = t[(t['交易类型']==1)].loc[:,['用户标识', '流水时间', '交易金额']].rename(index = str, columns={'交易金额':'支出'})
t_income_expend = pd.merge(t_income, t_expend, how='inner', on = ['用户标识', '流水时间'])
t_income_expend['收入支出差'] = t_income_expend['收入'] - t_income_expend['支出']
t_income_minus_expend_sum = t_income_expend.loc[:, ['用户标识','收入支出差']].groupby(["用户标识"],as_index=False)

x5=t_income_minus_expend_sum['收入支出差'].agg({'放款前收入支出差笔数' : 'count',
                       '放款前收入支出差总计':'sum',
                       '放款前收入支出差最大值':'min',
                       '放款前收入支出差最小值':'max',
                       '放款前收入支出差平均值':'mean',
                       '放款前收入支出差中位数':'median',
                       '放款前收入支出差方差':'var',
                       '放款前收入支出差标准差':'std'}) 
x5

#t2 = t[(t['交易类型']==1)]

NameError: name 'loantime' is not defined

In [13]:
import time
time_now = datetime.datetime.now()
cost_time = time.time()-start_time
filename = datetime.datetime.strftime(time_now,'%Y%m%d_%H%M%S')


NameError: name 'datetime' is not defined

# 5.overdue

In [14]:
overdue = pd.read_csv(OVERDUE_TRAIN_FILE,header=0,names=['用户标识','放款时间'])
overdue[overdue['放款时间']==0].shape[0]/overdue[overdue['放款时间']==1].shape[0]


FileNotFoundError: File b'../train/overdue_train.csv' does not exist

In [15]:
def testPrint(a, b):
    return (a,b)

(c,d) = testPrint(1,2)
c


1